In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
resistivity = 10.6 * 0.0001 #10.6 µΩ cm https://iopscience.iop.org/article/10.1088/1742-6596/100/8/082006/pdf#:~:text=The%20bulk%20resistivity%20of%20Pt,results%20in%20higher%20film%20resistance.
thickness = 300 * 10**(-9) #300 nm
rho_medium = 1/0.0042 #condutivity of grey matter is 0.0042 S/m
R_CT = None #let's assume inert electrode coating (i.e. titanium nitride)
L_track = 3.5 * 0.01 #~3,5cm probe length
W_track = 0.5 * 0.001 #we assume half a millimeter width for now
diameter = 30 * 10**(-4) #30µm = 30 * e-4 cm

In [4]:
# Resistance of the track
R_sheet = resistivity / thickness
R_track = R_sheet * L_track/W_track
R_spread = rho_medium/(4*(diameter/2))

R_series = R_track + R_spread
print("R_series = ", R_series, "Ohm")

R_series =  287015.873015873 Ohm


In [5]:
# Neurons size
area = 350# * 10**(-6) µm^2
diameter = np.sqrt(area/np.pi) * 2
print(diameter)
volume = 350 #cm^2

21.11004122822376


In [6]:
# Neurons distances
density = 5000 #cells/mm^3
empty_vol = 1/density #mm^3
print(empty_vol) #mm^3
distance_sphere = (empty_vol * 3/4 / np.pi)**(1/3)#mm
distance_square = empty_vol**(1/3) #mm
distance_hex = (empty_vol/(2*np.sqrt(2)))**(1/3) #mm
print(distance_sphere, distance_hex, distance_square)

0.0002
0.03627831678597811 0.04135185542000138 0.05848035476425733


In [7]:
# Electrodes parameters
side_length = 40 * 10**(-4) #40µm = 40 * e-4 cm
GSA = side_length**2 # electrode surface area
print('GSA = ', GSA, 'cm^2')
print('GSA = ', GSA*10**8, 'µm^2')

GSA =  1.6e-05 cm^2
GSA =  1600.0 µm^2


Check for charge density within safety limits
What is the shape of the stimulation --> to determine the charge injected
        if rectangle shape: what current? for how long?

In [8]:
# Pulse parameters:
# according to this study: Different effects of monophasic pulses and biphasic pulses applied by a bipolar stimulation electrode in the rat hippocampal CA1 region
# biphasic current pulse cathodic first
width = 0.1e-3 #0.1 ms # maybe go up to 300µs as in the paper
amplitude= 150e-6 #150µA
interphase_dwell = 100e-6 #100µs
Q_inj = (width * amplitude)*10**6 #µC
print('Q_inj :', Q_inj)
print('log10(Q_inj) :', np.log10(Q_inj))

Q_inj : 0.015
log10(Q_inj) : -1.8239087409443189


In [9]:
# Charge density
Charge_density = Q_inj/GSA
print('GSA = ', GSA)
print('Charge density = ', Charge_density) #expressind in µC/cm2

GSA =  1.6e-05
Charge density =  937.5


In [10]:
#Shannon plot for stimulation safety
k = 1.75 

# Check if the charge density is within the Shannon limit
def Within_Shannon_limit(Q, A, k):
    return np.log(Q/A) <= k - np.log(Q)

# Compute Shannon limit given a GSA and a k value
def Shannon_limit(GSA, k):
    return np.sqrt(np.exp(k) * GSA)

# Compute the pulse width or amplitude given the other and the charge density
def Shannon_limit_pulse(Q, param):
    return Q*10**(-6)/param

In [11]:
Within_Shannon_limit(Q_inj, GSA, k) #might be wrong because we consider charge density heare and not charge capacity

False

In [18]:
Q_max = Shannon_limit(GSA, k)
print('Q_max =', Q_max, 'µC')
print('Q_max =', Q_max*10**3, 'nC')

Q_max = 0.009595501175868392 µC
Q_max = 9.595501175868392 nC


In [17]:
# If we have a pulse width of 0.1 ms, what is the maximum amplitude we can use?
width = 0.1e-3 # generally between 50e-6 and 4e-3
max_amp = Shannon_limit_pulse(Q_max, width)
print('max_amp =', max_amp*10**3, 'mA')

# If we have a pulse amplitude of 150 µA, what is the maximum pulse width we can use?
amplitude= 150e-6
max_width = Shannon_limit_pulse(Q_max, amplitude)
print('max_width =', max_width*10**6, 'µs')

max_amp = 0.09595501175868393 mA
max_width = 63.97000783912262 µs


In [21]:
# Values for existing electrodes
pulse_width_DBS = 100e-6 #100µs
Qinj_DBS = 200e-9 #200nC
pulse_width_intracortical = 200e-6 #200µs
Q_inj_intracortical = 4.6e-9 #4.6nC
pulse_width_hearing_cat = 150e-6 #150µs
Q_inj_hearing_cat = 1.5e-9 #1.5nC
pulse_width_intraspinal_cat = 100e-6 #100µs
Q_inj_intraspinal_cat = 9e-9 #9nC

# Get maximum amplitude for existing electrodes
max_amp_DBS = Shannon_limit_pulse(Qinj_DBS, pulse_width_DBS)
print('max_amp_DBS =', max_amp_DBS*10**6, 'µA')
max_amp_intracortical = Shannon_limit_pulse(Q_inj_intracortical, pulse_width_intracortical)
print('max_amp_intracortical =', max_amp_intracortical*10**6, 'µA')
max_amp_hearing_cat = Shannon_limit_pulse(Q_inj_hearing_cat, pulse_width_hearing_cat)
print('max_amp_hearing_cat =', max_amp_hearing_cat*10**6, 'µA')
max_amp_intraspinal_cat = Shannon_limit_pulse(Q_inj_intraspinal_cat, pulse_width_intraspinal_cat)
print('max_amp_intraspinal_cat =', max_amp_intraspinal_cat*10**6, 'µA')

max_amp_DBS = 0.0019999999999999996 µA
max_amp_intracortical = 2.2999999999999997e-05 µA
max_amp_hearing_cat = 1e-05 µA
max_amp_intraspinal_cat = 8.999999999999999e-05 µA


In [ ]:
# Get eletrode surface area for existing electrodes
def GSA_from_charge_density(charge_density, Q):
    return Q/charge_density

In [14]:
# Test with other values
CIC_theoretical = 100*10**(-6) #maximum value for platinum would be 100 µm/cm2
Qinj_theor = CIC_theoretical * GSA #

In [15]:
Within_Shannon_limit(Qinj_theor, GSA, k) #considering charge injection capacity

True